<a href="https://colab.research.google.com/github/NoureldinAyman/Drug-Recommendation/blob/nour/Another_copy_of_ANN_Nour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Personalized Drug Recommendation

## Dataset
The dataset is MIMIC-IV


It is separated into 4 modules:
- [hosp](https://mimic.mit.edu/docs/iv/modules/hosp) - hospital level data for patients: labs, micro, and electronic medication administration
- [icu](https://mimic.mit.edu/docs/iv/modules/icu) - ICU level data. These are the event tables, and are identical in structure to MIMIC-III (chartevents, etc)
- [ed](https://mimic.mit.edu/docs/iv/modules/ed) - data from the emergency department
- [note](https://mimic.mit.edu/docs/iv/modules/note) - deidentified free-text clinical notes

Since I'll only be working on tabular data, I will exclude `note`.

## Data preprocessing


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setting the seed for reproducibility

In [13]:
np.random.seed(42)
sample_frac = 0.10

Tables used:
1. omr table
  - The Online Medical Record (OMR) table contains miscellaneous information from the EHR.
2. admissions table
	- Detailed information about hospital stays.
3. d_labitems
	- Dimension table for labevents provides a description of all lab items.
4. diagnoses_icd
	- Billed ICD-9/ICD-10 diagnoses for hospitalizations.
5. labevents
	- Laboratory measurements sourced from patient derived specimens.
6. microbiologyevents
	- Microbiology cultures.
7. patients table
	- Patients' gender, age, and date of death if information exists.
8. prescriptions
	- Prescribed medications.

### Loading datasets

In [14]:
# Load datasets with relevant columns
admissions = pd.read_csv("/content/drive/MyDrive/Data/hosp/admissions.csv.gz",
                         compression="gzip",
                         usecols=["subject_id", "hadm_id", "admittime", "admission_type",
                                  "admission_location", "insurance", "language", "marital_status", "race"],
                         nrows=100)

patients = pd.read_csv("/content/drive/MyDrive/Data/hosp/patients.csv.gz",
                       compression="gzip",
                       usecols=["subject_id", "gender", "anchor_age"],
                       nrows=100)

omr = pd.read_csv("/content/drive/MyDrive/Data/hosp/omr.csv.gz",
                  compression="gzip",
                  usecols=["subject_id", "chartdate", "seq_num", "result_name", "result_value"],
                  nrows=100)

labevents = pd.read_csv("/content/drive/MyDrive/Data/hosp/labevents.csv.gz",
                        compression="gzip",
                        usecols=["subject_id", "hadm_id", "itemid", "charttime", "valuenum", "valueuom", "flag"],
                        nrows=100)

# d_labitems = pd.read_csv("/content/drive/MyDrive/Data/hosp/d_labitems.csv.gz",
#                          compression="gzip",
#                          usecols=["itemid", "label"])

microbiologyevents = pd.read_csv("/content/drive/MyDrive/Data/hosp/microbiologyevents.csv.gz",
                                 compression="gzip",
                                 usecols=["subject_id", "hadm_id", "charttime", "spec_type_desc", "org_name"],
                                 nrows=100)

diagnoses_icd = pd.read_csv("/content/drive/MyDrive/Data/hosp/diagnoses_icd.csv.gz",
                            compression="gzip",
                            usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
                            nrows=100)

prescriptions = pd.read_csv("/content/drive/MyDrive/Data/hosp/prescriptions.csv.gz",
                            compression="gzip",
                            usecols=["subject_id", "hadm_id", "drug", "dose_val_rx", "dose_unit_rx",
                                     "starttime", "stoptime"],
                            nrows=100)

In [15]:
print(f"admissions shape: {admissions.shape}")
print(f"patients shape: {patients.shape}")
print(f"omr shape: {omr.shape}")
print(f"labevents shape: {labevents.shape}")
# print(f"d_labitems shape: {d_labitems.shape}")
print(f"microbiologyevents shape: {microbiologyevents.shape}")
print(f"diagnoses_icd shape: {diagnoses_icd.shape}")
print(f"prescriptions shape: {prescriptions.shape}")

admissions shape: (100, 9)
patients shape: (100, 3)
omr shape: (100, 5)
labevents shape: (100, 7)
microbiologyevents shape: (100, 5)
diagnoses_icd shape: (100, 4)
prescriptions shape: (100, 7)


In [27]:
database = {
    "admissions": admissions,
    "patients": patients,
    "omr": omr,
    "labevents": labevents,
    "microbiologyevents": microbiologyevents,
    "diagnoses_icd": diagnoses_icd,
    "prescriptions": prescriptions
}

for table_name, df in database.items():
    print(f"{table_name} has columns: {df.columns.tolist()}")


admissions has columns: ['subject_id', 'hadm_id', 'admittime', 'admission_type', 'admission_location', 'insurance', 'language', 'marital_status', 'race']
patients has columns: ['subject_id', 'gender', 'anchor_age']
omr has columns: ['subject_id', 'chartdate', 'seq_num', 'result_name', 'result_value']
labevents has columns: ['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum', 'valueuom', 'flag']
microbiologyevents has columns: ['subject_id', 'hadm_id', 'charttime', 'spec_type_desc', 'org_name']
diagnoses_icd has columns: ['subject_id', 'hadm_id', 'icd_code', 'icd_version']
prescriptions has columns: ['subject_id', 'hadm_id', 'starttime', 'stoptime', 'drug', 'dose_val_rx', 'dose_unit_rx']


### Merging the datasets

Starting from the base admissions table, I'm going to:
- Create a copy of the base admissions.
- Merge admissions copy with patients on `subject_id` with left join.
- Merge with diagnoses on `subject_id`.
- Merge with prescriptions on `subject_id` and `hadm_id`.
- Merge with labevents on `subject_id` and `hadm_id`.
- Merge with microbiologyevents on `subject_id` and `hadm_id`.

All merges are done with left join to preserve the records in the admissions table.

In [16]:
# Start with admissions as the base table
dataset = admissions.copy()

# Merge with patients using subject_id
dataset = dataset.merge(patients, on='subject_id', how='left')

# Merge with diagnoses_icd using subject_id and hadm_id
dataset = dataset.merge(diagnoses_icd, on=['subject_id', 'hadm_id'], how='left')

# Merge with prescriptions using subject_id and hadm_id
dataset = dataset.merge(prescriptions, on=['subject_id', 'hadm_id'], how='left')

# Merge with labevents using subject_id and hadm_id
dataset = dataset.merge(labevents, on=['subject_id', 'hadm_id'], how='left')

# Merge with microbiologyevents using subject_id and hadm_id
dataset = dataset.merge(microbiologyevents, on=['subject_id', 'hadm_id'], how='left')

# Merge with omr using subject_id
dataset = dataset.merge(omr, on='subject_id', how='left')

Validate the result


In [17]:
print("Shape of merged data:", dataset.shape)
print("Missing values in key columns:")
print(dataset[['subject_id', 'hadm_id']].isnull().sum())

Shape of merged data: (74381, 30)
Missing values in key columns:
subject_id    0
hadm_id       0
dtype: int64


In [18]:
dataset.columns

Index(['subject_id', 'hadm_id', 'admittime', 'admission_type',
       'admission_location', 'insurance', 'language', 'marital_status', 'race',
       'gender', 'anchor_age', 'icd_code', 'icd_version', 'starttime',
       'stoptime', 'drug', 'dose_val_rx', 'dose_unit_rx', 'itemid',
       'charttime_x', 'valuenum', 'valueuom', 'flag', 'charttime_y',
       'spec_type_desc', 'org_name', 'chartdate', 'seq_num', 'result_name',
       'result_value'],
      dtype='object')

In [19]:
dataset.shape

(74381, 30)

In [20]:
dataset.isnull().sum()

,0
subject_id,0
hadm_id,0
admittime,0
admission_type,0
admission_location,0
insurance,4
language,0
marital_status,1
race,0
gender,0


In [22]:
dataset["itemid"]

,itemid
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
74376,NaN
74377,NaN
74378,NaN
74379,NaN


In [21]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74381 entries, 0 to 74380
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   subject_id          74381 non-null  int64  
 1   hadm_id             74381 non-null  int64  
 2   admittime           74381 non-null  object 
 3   admission_type      74381 non-null  object 
 4   admission_location  74381 non-null  object 
 5   insurance           74377 non-null  object 
 6   language            74381 non-null  object 
 7   marital_status      74380 non-null  object 
 8   race                74381 non-null  object 
 9   gender              74381 non-null  object 
 10  anchor_age          74381 non-null  int64  
 11  icd_code            74297 non-null  object 
 12  icd_version         74297 non-null  float64
 13  starttime           74252 non-null  object 
 14  stoptime            74252 non-null  object 
 15  drug                74252 non-null  object 
 16  dose

### Encoding/Scaling

In [ ]:
numerical_cols = dataset.select_dtypes(include=["number"]).columns
print("Numerical columns:")
print(numerical_cols)

categorical_cols = dataset.select_dtypes(include=["object", "category"]).columns
print("Categorical columns:")
print(categorical_cols)

Numerical columns:
Index(['subject_id', 'hadm_id', 'anchor_age', 'icd_version', 'itemid',
       'valuenum', 'seq_num'],
      dtype='object')
Categorical columns:
Index(['admittime', 'admission_type', 'admission_location', 'insurance',
       'language', 'marital_status', 'race', 'gender', 'icd_code', 'starttime',
       'stoptime', 'drug', 'dose_val_rx', 'dose_unit_rx', 'charttime_x',
       'valueuom', 'flag', 'charttime_y', 'spec_type_desc', 'org_name',
       'chartdate', 'result_name', 'result_value'],
      dtype='object')


In [ ]:
for cat_col in categorical_cols:
    uniques = dataset[cat_col].unique()
    print(f"{cat_col} has {len(uniques)} unique values")
    print(f"Examples: {uniques[:15]}")
    print("-------------------------------------------")

admittime has 100 unique values
Examples: ['2180-05-06 22:23:00' '2180-06-26 18:27:00' '2180-08-05 23:44:00'
 '2180-07-23 12:35:00' '2160-03-03 23:16:00' '2160-11-21 01:56:00'
 '2160-12-28 05:11:00' '2163-09-27 23:17:00' '2181-11-15 02:05:00'
 '2183-09-18 18:10:00' '2163-08-20 01:42:00' '2192-11-30 01:25:00'
 '2151-03-18 03:28:00' '2189-10-15 10:30:00' '2143-12-23 14:55:00']
-------------------------------------------
admission_type has 8 unique values
Examples: ['URGENT' 'EW EMER.' 'EU OBSERVATION' 'OBSERVATION ADMIT'
 'SURGICAL SAME DAY ADMISSION' 'AMBULATORY OBSERVATION' 'DIRECT EMER.'
 'DIRECT OBSERVATION']
-------------------------------------------
admission_location has 8 unique values
Examples: ['TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM' 'WALK-IN/SELF REFERRAL'
 'PHYSICIAN REFERRAL' 'PROCEDURE SITE' 'CLINIC REFERRAL'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PACU']
-------------------------------------------
insurance has 4 unique values
Examples: ['Medicaid' nan 'Medicare' 'Pr

Chart times are represented in year-month-day using strings. I will convert them them to YYYYMM in integers. Where Y is year and M is month.

In [ ]:
# Identify time-related columns
time_cols = [col for col in dataset.columns if 'time' in col.lower() or 'date' in col.lower()]
print("Time columns:", time_cols)

# Check original data
print("Original sample of time columns:")
for col in time_cols:
    print(f"{col}: {dataset[col].head().tolist()}")

# Convert to datetime and then to YYYYMM
for col in time_cols:
    # Convert to string to avoid type issues
    dataset[col] = dataset[col].astype(str)
    # Convert to datetime, coerce invalid values to NaT
    dataset[col] = pd.to_datetime(dataset[col], errors="coerce")
    # Convert to YYYYMM, fill NaT with 0, cast to int
    dataset[col] = (dataset[col].dt.year * 100 + dataset[col].dt.month).fillna(0).astype(int)

# Check results
print("Sample of converted columns:")
print(dataset[time_cols].head())

Time columns: ['admittime', 'starttime', 'stoptime', 'charttime_x', 'charttime_y', 'chartdate']
Original sample of time columns:
admittime: ['2180-05-06 22:23:00', '2180-05-06 22:23:00', '2180-05-06 22:23:00', '2180-05-06 22:23:00', '2180-05-06 22:23:00']
starttime: ['2180-05-08 08:00:00', '2180-05-08 08:00:00', '2180-05-08 08:00:00', '2180-05-08 08:00:00', '2180-05-08 08:00:00']
stoptime: ['2180-05-07 22:00:00', '2180-05-07 22:00:00', '2180-05-07 22:00:00', '2180-05-07 22:00:00', '2180-05-07 22:00:00']
charttime_x: [nan, nan, nan, nan, nan]
charttime_y: ['2180-05-07 00:10:00', '2180-05-07 00:10:00', '2180-05-07 00:10:00', '2180-05-07 00:10:00', '2180-05-07 00:10:00']
chartdate: ['2180-04-27', '2180-04-27', '2180-05-07', '2180-05-07', '2180-05-07']
Sample of converted columns:
   admittime  starttime  stoptime  charttime_x  charttime_y  chartdate
0     218005     218005    218005            0       218005     218004
1     218005     218005    218005            0       218005     218004

One hot encoding categorical columns except for admission type as it is ordinal.

"admission_type is useful for classifying the urgency of the admission. There are 9 possibilities: ‘AMBULATORY OBSERVATION’, ‘DIRECT EMER.’, ‘DIRECT OBSERVATION’, ‘ELECTIVE’, ‘EU OBSERVATION’, ‘EW EMER.’, ‘OBSERVATION ADMIT’, ‘SURGICAL SAME DAY ADMISSION’, ‘URGENT’."

In [ ]:
filtered_rows = dataset["admission_type"]

filtered_rows.head()

,admission_type
0,URGENT
1,URGENT
2,URGENT
3,URGENT
4,URGENT


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# One hot encode all categorical columns except for admission type as it is ordinal.
one_hot_cols = [col for col in categorical_cols if col != 'admission_type']

admission_order = ['AMBULATORY OBSERVATION', 'DIRECT EMER.', 'DIRECT OBSERVATION','ELECTIVE',
                   'EU OBSERVATION', 'EW EMER.', 'OBSERVATION ADMIT', 'SURGICAL SAME DAY ADMISSION', 'URGENT']
ordinal_encoder = OrdinalEncoder(categories=[admission_order])
dataset['admission_type_encoded'] = ordinal_encoder.fit_transform(dataset[['admission_type']]).ravel()

dataset = dataset.drop('admission_type', axis=1)

dataset_encoded = pd.get_dummies(dataset, columns=one_hot_cols)

In [ ]:
filtered_rows = dataset["admission_type_encoded"]

filtered_rows.head()

,admission_type_encoded
0,8.0
1,8.0
2,8.0
3,8.0
4,8.0


See the max and min of numerical features

In [ ]:
# # Print min and max for each numerical column
# for col in numerical_cols.columns:
#     print(f"{col}: min = {numerical_cols[col].min()}, max = {numerical_cols[col].max()}")


In [ ]:
dataset_encoded.drop(["subject_id", "hadm_id"], axis=1, inplace=True)

In [ ]:
dataset_encoded.shape

(74381, 402)

Scaling numerical data

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# dataset_scaled = dataset_encoded.copy()
# dataset_scaled[numerical_cols] = scaler.fit_transform(dataset_scaled[numerical_cols])
# print(dataset_scaled[numerical_cols].agg(["mean","std"]).round(3))